# Step 1: Load the Dataset


In [2]:
import pandas as pd
file_path = 'space_traffic.csv'
data = pd.read_csv(file_path)
data.head()


,Timestamp,Location,Object_Type,Traffic_Density,Peak_Time
0,21-10-2024 21:00,Lagrange Point L2,Space Station,17,15:00
1,11-10-2024 05:00,Orbit LEO,Satellite,21,15:00
2,29-10-2024 13:00,Orbit LEO,Space Station,88,6:00
3,24-10-2024 08:00,Mars Transfer Orbit,Scientific Probe,65,8:00
4,23-10-2024 17:00,Lagrange Point L1,Space Station,9,6:00


# Step 2: Extract Features from Timestamp

In [3]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data['Year'] = data['Timestamp'].dt.year
data['Month'] = data['Timestamp'].dt.month
data['Day'] = data['Timestamp'].dt.day
data['Hour'] = data['Timestamp'].dt.hour
data[['Timestamp', 'Year', 'Month', 'Day', 'Hour']].head()


C:\Users\sharvani dhulipala\AppData\Local\Temp\ipykernel_11200\643591398.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Timestamp'] = pd.to_datetime(data['Timestamp'])


,Timestamp,Year,Month,Day,Hour
0,2024-10-21 21:00:00,2024,10,21,21
1,2024-10-11 05:00:00,2024,10,11,5
2,2024-10-29 13:00:00,2024,10,29,13
3,2024-10-24 08:00:00,2024,10,24,8
4,2024-10-23 17:00:00,2024,10,23,17


Temporal features can provide insights into patterns over time, such as seasonality or trends.

# Step 3: Convert Peak Time into Numeric Representation

In [4]:
data['Peak_Time_Hour'] = pd.to_datetime(data['Peak_Time'], format='%H:%M').dt.hour
data[['Peak_Time', 'Peak_Time_Hour']].head()


,Peak_Time,Peak_Time_Hour
0,15:00,15
1,15:00,15
2,6:00,6
3,8:00,8
4,6:00,6


Numerical representation of peak time enables the model to analyze its effect on traffic density.

# Step 4: Encode Location

In [5]:
data['Location_Encoded'] = data['Location'].astype('category').cat.codes
data[['Location', 'Location_Encoded']].head()


,Location,Location_Encoded
0,Lagrange Point L2,1
1,Orbit LEO,4
2,Orbit LEO,4
3,Mars Transfer Orbit,2
4,Lagrange Point L1,0


Machine learning models work better with numerical inputs. Encoding simplifies categorical features.

# Step 5: One-Hot Encode Object_Type

In [6]:
object_type_encoded = pd.get_dummies(data['Object_Type'], prefix='Object_Type')
data = pd.concat([data, object_type_encoded], axis=1)
data.head()


,Timestamp,Location,Object_Type,Traffic_Density,Peak_Time,Year,Month,Day,Hour,Peak_Time_Hour,Location_Encoded,Object_Type_Asteroid Mining Ship,Object_Type_Manned Spacecraft,Object_Type_Satellite,Object_Type_Scientific Probe,Object_Type_Space Debris,Object_Type_Space Station
0,2024-10-21 21:00:00,Lagrange Point L2,Space Station,17,15:00,2024,10,21,21,15,1,False,False,False,False,False,True
1,2024-10-11 05:00:00,Orbit LEO,Satellite,21,15:00,2024,10,11,5,15,4,False,False,True,False,False,False
2,2024-10-29 13:00:00,Orbit LEO,Space Station,88,6:00,2024,10,29,13,6,4,False,False,False,False,False,True
3,2024-10-24 08:00:00,Mars Transfer Orbit,Scientific Probe,65,8:00,2024,10,24,8,8,2,False,False,False,True,False,False
4,2024-10-23 17:00:00,Lagrange Point L1,Space Station,9,6:00,2024,10,23,17,6,0,False,False,False,False,False,True


One-hot encoding ensures each category is represented as an independent feature, avoiding ordinal relationships.

# Step 6: Categorize Traffic Density

In [7]:
def categorize_density(density):
    if density < 30:
        return 'Low'
    elif 30 <= density < 70:
        return 'Medium'
    else:
        return 'High'
data['Traffic_Density_Category'] = data['Traffic_Density'].apply(categorize_density)
data[['Traffic_Density', 'Traffic_Density_Category']].head()


,Traffic_Density,Traffic_Density_Category
0,17,Low
1,21,Low
2,88,High
3,65,Medium
4,9,Low


Categorizing traffic density helps in understanding and modeling traffic patterns more intuitively.

# Step 7: Drop Irrelevant Columns

In [8]:
processed_data = data.drop(columns=['Timestamp', 'Peak_Time', 'Location', 'Object_Type'])
processed_data.to_csv('processed_space_traffic.csv', index=False)
processed_data.head()


,Traffic_Density,Year,Month,Day,Hour,Peak_Time_Hour,Location_Encoded,Object_Type_Asteroid Mining Ship,Object_Type_Manned Spacecraft,Object_Type_Satellite,Object_Type_Scientific Probe,Object_Type_Space Debris,Object_Type_Space Station,Traffic_Density_Category
0,17,2024,10,21,21,15,1,False,False,False,False,False,True,Low
1,21,2024,10,11,5,15,4,False,False,True,False,False,False,Low
2,88,2024,10,29,13,6,4,False,False,False,False,False,True,High
3,65,2024,10,24,8,8,2,False,False,False,True,False,False,Medium
4,9,2024,10,23,17,6,0,False,False,False,False,False,True,Low
